<a href="https://colab.research.google.com/github/lukaszplust/Projects/blob/main/Btree_DB_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
class Record:
    def __init__(self, key, data):
        self.key = key
        self.data = data

    def __repr__(self):
        return f"Record({self.key}, {self.data})"


    # to_bytes - metoda konwertująca obiekt Record na bajty, co jest przydatne do zapisywania danych w pliku

    def to_bytes(self):
        # konwertuje klucz key na 4-bajtowy zapis w porządku big-endian
        key_bytes = self.key.to_bytes(4, byteorder='big')
        # koduje dane data na bajty (domyślnie przy użyciu kodowania UTF-8)
        data_bytes = self.data.encode()
        # obliczam długość data_bytes i konwertuje ją na 4-bajtowy zapis w porządku big-endian
        length_bytes = len(data_bytes).to_bytes(4, byteorder='big')
        # zwracam jedną sekwencję bajtów
        return key_bytes + length_bytes + data_bytes

    # dekorator oznaczający, że metoda nie zależy od instancji klasy i może być wywoływana na poziomie klasy
    @staticmethod
    def from_bytes(byte_data):
        # konwertuje pierwsze 4 bajty byte_data na liczbę całkowitą w porządku big-endian
        key = int.from_bytes(byte_data[:4], byteorder='big')
        # konwertuje kolejne 4 bajty byte_data na liczbę całkowitą, reprezentującą długość danych
        length = int.from_bytes(byte_data[4:8], byteorder='big')
        # dekoduje bajty reprezentujące dane z byte_data na string
        data = byte_data[8:8+length].decode()
        # tworze i zwracam nowy obiekt Record z kluczem i danymi odczytanymi z bajtów
        return Record(key, data)



class ReadBuffer:
    def __init__(self, path, buffer_size=4):
        # przechowuje ścieżkę do pliku
        self.path = path
        # ustalam rozmiar bufora, czyli liczbę rekordów, które będą trzymane w pamięci
        self.buffer_size = buffer_size
        # inicjalizuje pustą listę bufora
        self.buffer = []
        # otwieram plik w trybie binarnym do odczytu
        self.file = open(self.path, "rb")
        # ustalam początkową pozycję w buforze
        self.position = 0
        # inicjalizuje licznik operacji odczytu z dysku
        self.disk_reads_count = 0

    # metoda odczytująca następny rekord z bufora
    def read_next(self):
        # sprawdzam, czy aktualna pozycja jest poza zakresem bufora
        if self.position >= len(self.buffer):
            # jeśli bufor jest pusty lub trzeba go uzupełnić, wywołuje metodę _fill_buffer
            self._fill_buffer()
        # sprawdzam, czy bufor jest pusty po jego uzupełnieniu
        if not self.buffer:
            # jeśli bufor jest pusty, zwraca None
            return None
        # pobiera rekord z aktualnej pozycji w buforze
        record = self.buffer[self.position]
        # zwiększam pozycję w buforze
        self.position += 1
        # zwracam pobrany rekord
        return record

    # metoda (prywatna) uzupełniająca bufor danymi z pliku
    def _fill_buffer(self):
        # czyszcze bufor
        self.buffer = []
        # resetuje pozycję w buforze
        self.position = 0
        # iteruje przez rozmiar bufora
        for _ in range(self.buffer_size):
            # odczytuje 8 bajtów z pliku
            bytes_record = self.file.read(8)
            # sprawdzam, czy odczytano dane
            if not bytes_record:
                # jeśli nie, kończe pętlę
                break
            # odczytuje długość danych z bajtów
            length = int.from_bytes(bytes_record[4:8], byteorder='big')
            # odczytuje dane o długości length
            data = self.file.read(length)
            # tworze rekord z bajtów i dodaje go do bufora
            self.buffer.append(Record.from_bytes(bytes_record + data))
        # zwiększam licznik operacji odczytu
        self.disk_reads_count += 1

    # metoda zamykająca plik
    def close(self):
        self.file.close()


class WriteBuffer:
    def __init__(self, path, buffer_size=4):
        # przechowuje ścieżkę do pliku
        self.path = path
        # ustalam rozmiar bufora, czyli liczbę rekordów, które będą trzymane w pamięci
        self.buffer_size = buffer_size
        # inicjalizuje pustą listę bufora
        self.buffer = []
        # inicjalizuje licznik operacji zapisu na dysku
        self.disk_writes_count = 0
    # metoda zapisująca rekord do bufora
    def write(self, record):
        # dodaje rekord do bufora
        self.buffer.append(record)
        # sprawdzam, czy bufor osiągnął rozmiar buffer_size
        if len(self.buffer) >= self.buffer_size:
            # jeśli bufor jest pełny, zapisuje jego zawartość na dysku
            self.flush()

    # metoda zapisująca zawartość bufora do pliku
    def flush(self):
        # otwieram plik w trybie dodawania binarnego ("ab")
        with open(self.path, "ab") as file:
            # iteruje przez rekordy w buforze
            for record in self.buffer:
                # zapisuje każdy rekord jako bajty do pliku
                file.write(record.to_bytes())
        # zwiększam licznik operacji zapisu
        self.disk_writes_count += 1
        # czyszcze bufor po zapisaniu
        self.buffer = []

    # metoda zamykająca bufor, zapisując pozostałe rekordy
    def close(self):
        # sprawdzam, czy w buforze są jakieś dane
        if self.buffer:
            # jeśli są dane, to zapisuje je do pliku
            self.flush()


class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t  # Minimalny stopień (t)
        self.leaf = leaf  # Czy jest liściem
        self.keys = []  # Lista kluczy
        self.children = []  # Lista dzieci (węzłów)

    def __repr__(self):
        return f'BTreeNode(keys={self.keys}, leaf={self.leaf})'

    def find_key(self, k):
        idx = 0
        while idx < len(self.keys) and self.keys[idx] < k:
            idx += 1
        return idx

    def insert_non_full(self, k):
        i = len(self.keys) - 1
        if self.leaf:
            self.keys.append(None)
            while i >= 0 and self.keys[i] > k:
                self.keys[i + 1] = self.keys[i]
                i -= 1
            self.keys[i + 1] = k
        else:
            while i >= 0 and self.keys[i] > k:
                i -= 1
            i += 1
            if len(self.children[i].keys) == 2 * self.t - 1:
                self.split_child(i)
                if self.keys[i] < k:
                    i += 1
            self.children[i].insert_non_full(k)

    def split_child(self, i):
        t = self.t
        y = self.children[i]
        z = BTreeNode(t, y.leaf)
        self.children.insert(i + 1, z)
        self.keys.insert(i, y.keys[t - 1])
        z.keys = y.keys[t:(2 * t) - 1]
        y.keys = y.keys[0:t - 1]
        if not y.leaf:
            z.children = y.children[t:(2 * t)]
            y.children = y.children[0:t - 1]

class BTree:
    def __init__(self, t, path, buffer_size=4):
        self.root = BTreeNode(t, leaf=True)
        self.t = t
        self.read_operations = 0
        self.write_operations = 0
        self.path = path
        self.read_buffer = ReadBuffer(path, buffer_size)
        self.write_buffer = WriteBuffer(path, buffer_size)
        self.buffer_size = buffer_size
        self.max_keys = 2 * t - 1  # Maksymalna liczba kluczy w węźle

    # 1. Wstawianie Rekordu

    def insert(self, record):
        if self.search(record.key) is not None:
            print(f'Already exists: {record.key}')
            return

        root = self.root
        if len(root.keys) == self.max_keys:
            temp = BTreeNode(self.t)
            self.root = temp
            temp.children.insert(0, root)
            self._split_child(temp, 0)
            self._insert_non_full(temp, record)
        else:
            self._insert_non_full(root, record)

        self.write_operations += 1
        self.write_buffer.write(record)

        # 6. Automatyczna Reorganizacja
        if len(self._get_all_keys(self.root)) > 2 * self.max_keys:
            self.reorganize()

    def _insert_non_full(self, node, record):
        node.insert_non_full(record.key)
        self.write_operations += 1

    def _split_child(self, node, i):
        node.split_child(i)
        self.write_operations += 1

    # 2. Odczyt Rekordu

    def read_record(self, key):
        self.read_operations += 1
        self.read_buffer.file.seek(0)
        while True:
            record = self.read_buffer.read_next()
            if record is None:
                break
            if record.key == key:
                return record
        return None

    # 3. Przeglądanie Całej Zawartości Pliku i Indeksu
    def traverse(self):
        self._traverse(self.root)
        print()



    def _traverse(self, node):
        for i in range(len(node.keys)):
            if not node.leaf:
                self._traverse(node.children[i])
            print(node.keys[i], end=' ')
        if not node.leaf:
            self._traverse(node.children[len(node.keys)])

    def search(self, k):
        self.read_operations += 1
        return self._search(self.root, k)

    def _search(self, node, k):
        i = node.find_key(k)
        if i < len(node.keys) and node.keys[i] == k:
            return (node.keys[i], node.children[i] if not node.leaf else None)

        if node.leaf:
            return None
        return self._search(node.children[i], k)

    # 4. Reorganizacja Pliku
    def reorganize(self):
        all_keys = self._get_all_keys(self.root)
        all_keys.sort()
        self.root = BTreeNode(self.t, leaf=True)
        self._bulk_load(all_keys)

    def _get_all_keys(self, node):
        keys = []
        if node.leaf:
            keys.extend(node.keys)
        else:
            for i in range(len(node.keys)):
                keys.extend(self._get_all_keys(node.children[i]))
                keys.append(node.keys[i])
            keys.extend(self._get_all_keys(node.children[len(node.keys)]))
        return keys

    def _bulk_load(self, keys):
        for key in keys:
            self.insert(Record(key, ''))

    # 5. Wyświetlanie Zawartości Pliku i Indeksu
    def print_structure(self):
        self._print_structure(self.root, 0)

    def _print_structure(self, node, level):
        print("Level", level, ":", node.keys)
        if not node.leaf:
            for child in node.children:
                self._print_structure(child, level + 1)

    def close_buffers(self):
        self.read_buffer.close()
        self.write_buffer.close()

# 7. Interaktywny i Plikowy Tryb Działania

def process_commands(btree, commands):
    for command in commands:
        parts = command.strip().split()
        if not parts:
            continue
        if parts[0] == 'insert':
            key = int(parts[1])
            data = parts[2]
            record = Record(key, data)
            btree.insert(record)
            print(f"Inserted {key}.")
            btree.print_structure()
        elif parts[0] == 'find':
            key = int(parts[1])
            found = btree.search(key)
            if found:
                record = btree.read_record(key)
                print(f"Found: {record}")
            else:
                print(f"Key {key} not found.")
        # Metoda traverse jest używana do wyświetlania wszystkich kluczy w B-drzewie w uporządkowany sposób (in-order traversal).
        # Dzięki temu można  zobaczyć wszystkie klucze w drzewie w kolejności rosnącej,
        # co jest pomocne do weryfikacji struktury drzewa i poprawności jego organizacji
        elif parts[0] == 'traverse':
            btree.traverse()
        elif parts[0] == 'stats':
            print(f"Read operations: {btree.read_operations}, Write operations: {btree.write_operations}")
        elif parts[0] == 'reorganize':
            btree.reorganize()
            print("Reorganized the B-tree.")
            btree.print_structure()
        elif parts[0] == 'print':
            btree.print_structure()
        else:
            print(f"Unknown command: {command}")

def read_commands_from_file(filename):
    with open(filename, 'r') as file:
        return file.readlines()

def main():
    path = 'test'
    buffer_size = 4
    btree = BTree(2, path, buffer_size)
    mode = input("Enter 'file' to read commands from a file or 'interactive' for interactive mode: ")
    if mode == 'file':
        filename = input("Enter the filename: ")
        commands = read_commands_from_file(filename)
        process_commands(btree, commands)
    elif mode == 'interactive':
        while True:
            command = input("Enter command: ")
            if command == 'exit':
                break
            process_commands(btree, [command])
    btree.close_buffers()

if __name__ == "__main__":
    main()

Enter 'file' to read commands from a file or 'interactive' for interactive mode: exit
